# Proyecto Final: Procesamiento de Lenguaje Natural

## Definición del proyecto

El objetivo de este proyecto es utilizar una Red Neuronal Recurrente usando LSTM (Long-Short Term Memory) para hacer traducción de Inglés a Italiano. En los datos de entrenamiento que se nos proporcionan tenemos un par de frases; la frase en inglés y su correspondiente traducción a italiano. 

## Referencias
 
 - Basile, Pierpaolo, et al. “Bi-Directional LSTM-CNNs-CRF for Italian Sequence Labeling.” Proceedings of the Fourth Italian Conference on Computational Linguistics CLiC-It 2017, 2017, pp. 18–23., doi:10.4000/books.aaccademia.2339.
 - Raval, Siraj. “Recurrent Neural Network - The Math of Intelligence (Week 5).” YouTube, YouTube, 19 July 2017, www.youtube.com/watch?v=BwmddtPFWtA.
 - Raval, Siraj. “LSTM Networks - The Math of Intelligence (Week 8).” YouTube, YouTube, 9 Aug. 2017, www.youtube.com/watch?v=9zhrxE5PQgY. 

